<a href="https://colab.research.google.com/github/BDonadelli/Codigos-em-financas/blob/main/extrato_Inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.google.com/spreadsheets/d/1ghinAy0SwtlEscBsWm9qSXedCZKuwfNQ7xJoiungC8s/edit#gid=657974412

In [ ]:
# # testa se roda pelo colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

import gspread
import pandas as pd


In [ ]:
# # se rodar pelo colab
if IN_COLAB  : # vai pegar o ultimo extrato da planilha de extratos e por num dataframe
    # from gspread_dataframe import get_as_dataframe, set_with_dataframe
    from google.colab import auth
    auth.authenticate_user()
    from google.auth import default
    credentials, _ = default()
    gc = gspread.authorize(credentials)
    planilha = gc.open('Extrato Inter 2023')
    df = pd.DataFrame(planilha.get_worksheet(1).get_all_records())
    # df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
    # df['Valor'] = df['Valor'].replace(regex={',': '.'}).astype(float)
    # df['Número do documento'] = df['Número do documento'].fillna(' ').astype(str)
    # # gc.session.close()
#########################################################################################################
# se rodar via google cloud
else:  # vai pegar o ultimo extrato do computador,  or num dataframe e escrever na planilha de extratos
    from oauth2client.service_account import ServiceAccountCredentials

    scope = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('destrincha-fatura.json',scope)
    gc = gspread.authorize(credentials)
    planilha = gc.open('Extrato Inter 2023')
    path='/home/yair/Dropbox/Desktop/Financas/extrato-inter.csv'
    df = pd.read_csv(path,
                 sep=';' , decimal=',' , thousands='.',
                 header = 4, index_col=False, usecols=[0,1,2,3],
                 parse_dates=[0],
                 dayfirst=True
                 )
    df.sort_values(by=['Data Lançamento'],inplace=True)
    # df.fillna('-',inplace=True)

    aba=str(input("mês ano (mmm YY): "))
    try :
        pagina = planilha.worksheet(aba)
        pagina.clear()
    except :
        pagina = planilha.add_worksheet(title=aba, rows=100, cols=20 , index= 1)
    # ajeita df
    df['Data Lançamento'] = df['Data Lançamento'].dt.strftime("%d/%m/%Y")
    # # escreve no gs
    pagina.update([df.columns.values.tolist()] + df.values.tolist())
    # gc.session.close()
    df['Data Lançamento'] = pd.to_datetime(df['Data Lançamento'], dayfirst=True)


#########################################################################################################
# df.info()

/home/yair/.local/lib/python3.7/site-packages/gspread/worksheet.py:1049: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  msg_deprecated="method signature will change to: 'Worksheet.update(value = [[]], range_name=)'"


In [ ]:
def limpa(espacos) :
  return " ".join(espacos.split())

def que_receita(string) :
  ''' associa a 'string' à chave do dicionário 'Despesas' ou 'Receitas' '''
  for t in receitas.keys() :
    for casos in receitas[t]:
      if casos in string:
        return t,casos
  else: return 'nao achei '+'"'+string+'"',None

def que_despesa(string) :
  ''' associa a 'string' à chave do dicionário 'Despesas' ou 'Receitas' '''
  for t in Despesas.keys() :
    for casos in Despesas[t]:
      if casos in string:
        return t,casos
  else: return 'nao achei '+'"'+string+'"',None

RF  = ['Cocal' , 'Jbs' , 'Direcional3']
RV  = ['CAML3' , 'BBDC3' , 'FLRY3' , 'SAPR11' , 'ITSA4' , 'TUPY3' , 'WIZC3' , 'CXSE3' , 'EGIE3' , 'TAEE11' , 'ABCB4',
       'SNEC34' , 'MSFT34' , 'A1MT34' , 'GOGL34' , 'COCA34' , 'KHCB34',
       'URPR11' , 'JURO11' , 'BIDB11',
       'OFSA3' , 'CPLE6' , 'BBAS3' , 'ENBR3' , 'BBDC4']
Fundos = ['Trigono' , 'Western' , 'Corporate' , 'V8']

marcador = ['Restaurante','Supermercado','Transportes','Carro','Eletrodoméstico','Educação',
          'Farmácia','Plano de saúde','Roupa','Lazer','Faxina','Aluguel','Condominio','Gás',
          'Luz','Telefone','Celular','TV','Casa','Mesada','Outros','Med/Dent/Psic','Sacolão',
          'Esportes','Internet','Lighia','Viagem','Presente' , 'Proventos' , 'Cashback']

In [ ]:
if IN_COLAB  :
    import requests
    url = str(input('pegar url no github (https://github.com/BDonadelli/Utilitarios/blob/main/INTERdict.json): '))
    f = requests.get(url).json()
    receitas = f[0]
    Despesas = f[1]
    Historico = f[2]
else :
    from INTERdict import *

In [ ]:
tipo = []
onde=[]
classe = []
marker = []

i=0
while i < len(df):
  #
  if df['Valor'].iloc[i] > 0  :
    #print(df['Histórico'].iloc[i]+'----'+que_receita(df['Histórico'].iloc[i])[0]+str(que_receita(df['Histórico'].iloc[i])[1]))
    tipo.append('C')

    aux1,aux2 = que_receita(df['Descrição'].iloc[i])

    if aux1 == 'Proventos' :
        if aux2 == 'Juros S/capital' : aux2 = 'JCP'
        for j in RV :
          if j in df['Descrição'].iloc[i].upper() : aux2=aux2+' '+j
        marker.append(aux1)

    if aux1 == 'B3' and aux2 == 'Liquidacao' :
        aux2='Venda de ações'

    if aux1 == 'Resgate' : aux2= aux1+' '+aux2

    if 'Cashback' in  df['Histórico'].iloc[i] :
        aux2 = 'Cashback'
        marker.append(aux2)

    if aux2 != None : onde.append(aux2)
    else: onde.append(df['Histórico'].iloc[i])
    classe.append('Recebido')
    i=i+1

  elif df['Valor'].iloc[i]< 0 :
    #print(df['Histórico'].iloc[i]+' ---- '+que_despesa(df['Histórico'].iloc[i]))
    tipo.append('D')
    aux,aux2 = que_despesa(df['Descrição'].iloc[i])

    if aux == 'Med/Dent/Psic':
      if aux2 == 'DENISE' : aux2 = 'Denise'

    if aux == 'Celular':
        if df['Data Lançamento'].iloc[i].day < 15 : aux2='Celular Gui'
        else : aux2='Celular Ja'

    if aux == 'B3' and aux2 == 'Liquidacao' :
        aux2='Compra de ações'

    if aux == 'Aplicação' and aux2 == 'Firf' :
       for fi in Fundos :
            if fi in df['Descrição'].iloc[i] : aux2 = aux+' '+fi



    onde.append(aux2)
    classe.append(aux)
    i=i+1
df['tipo'] = tipo
df['onde'] = onde
df['classe']=classe


In [ ]:
df

,Data Lançamento,Histórico,Descrição,Valor,tipo,onde,classe
23,2023-11-01,Crédito Evento B3,* Prov * Juros S/capital 270 Bbdc3,3.96,C,JCP BBDC3,Recebido
21,2023-11-01,Pagamento efetuado,COLEGIO TECNICO SINGULAR,-942.85,D,SINGULAR,Educação
20,2023-11-01,Transferência recebida,Jair Donadelli Junior,8000.00,C,Transferência recebida,Recebido
22,2023-11-01,Pix enviado,Ingrid Horodynski Donadelli,-100.00,D,Ingrid,Mesada
19,2023-11-02,Cashback,INTER PRE 10GB MENSAL,2.10,C,Cashback,Recebido
18,2023-11-02,Compra Inter Cel,Inter Pre 10gb Mensal,-30.00,D,Celular Gui,Celular
17,2023-11-03,Crédito B3,Liquidacao B3 * 31/10/2023,7.27,C,Liquidacao,Recebido
16,2023-11-03,Pix enviado,Jair Donadelli Junior,-100.00,D,Jair,Eu
15,2023-11-03,Cred Pontos Cashback Conta,Resgate Pontos,62.50,C,Cashback,Recebido
14,2023-11-04,Pix enviado,Carlos Alberto Figaro Bertin,-20.00,D,Bertin,Carro


In [ ]:
creditos = df[df.tipo=='C'].sort_values(by=['Data Lançamento'])
creditos

,Data Lançamento,Histórico,Descrição,Valor,tipo,onde,classe
23,2023-11-01,Crédito Evento B3,* Prov * Juros S/capital 270 Bbdc3,3.96,C,JCP BBDC3,Recebido
20,2023-11-01,Transferência recebida,Jair Donadelli Junior,8000.00,C,Transferência recebida,Recebido
19,2023-11-02,Cashback,INTER PRE 10GB MENSAL,2.10,C,Cashback,Recebido
17,2023-11-03,Crédito B3,Liquidacao B3 * 31/10/2023,7.27,C,Liquidacao,Recebido
15,2023-11-03,Cred Pontos Cashback Conta,Resgate Pontos,62.50,C,Cashback,Recebido
12,2023-11-06,Crédito Evento B3,* Prov * Dividendos 569 Cxse3,284.50,C,Dividendos CXSE3,Recebido
7,2023-11-07,Cred Pontos Cashback Conta,Resgate Pontos,57.50,C,Cashback,Recebido
5,2023-11-09,Crédito B3,Liquidacao B3 * 07/11/2023,934.93,C,Liquidacao,Recebido
4,2023-11-10,Crédito B3,Liquidacao B3 * 08/11/2023,1735.92,C,Liquidacao,Recebido


In [ ]:
debitos = df[df.tipo=='D'].sort_values(by=['Data Lançamento'])
debitos

,Data Lançamento,Histórico,Descrição,Valor,tipo,onde,classe
21,2023-11-01,Pagamento efetuado,COLEGIO TECNICO SINGULAR,-942.85,D,SINGULAR,Educação
22,2023-11-01,Pix enviado,Ingrid Horodynski Donadelli,-100.00,D,Ingrid,Mesada
18,2023-11-02,Compra Inter Cel,Inter Pre 10gb Mensal,-30.00,D,Celular Gui,Celular
16,2023-11-03,Pix enviado,Jair Donadelli Junior,-100.00,D,Jair,Eu
14,2023-11-04,Pix enviado,Carlos Alberto Figaro Bertin,-20.00,D,Bertin,Carro
13,2023-11-05,Pix enviado,Jair Donadelli Junior,-10.00,D,Jair,Eu
11,2023-11-06,Pagamento efetuado,Fatura cartão Inter,-5719.30,D,Fatura,Eu
9,2023-11-06,Aplicação,Cdb Pos Di Liq. Banco Inter Sa,-1700.00,D,Cdb,Aplicação
10,2023-11-06,Pix enviado,Jair Donadelli Junior,-5.00,D,Jair,Eu
8,2023-11-07,Pix enviado,Ingrid Horodynski Donadelli,-60.00,D,Ingrid,Mesada


In [ ]:
final = pd.concat([creditos,debitos])
final['Valor']=final['Valor'].apply(abs)
final['Data Lançamento'] = final['Data Lançamento'].dt.strftime('%d/%m/%Y')

In [ ]:
resumo = final[['Histórico','onde','Data Lançamento','classe','Valor','tipo']].copy()
resumo

,Histórico,onde,Data Lançamento,classe,Valor,tipo
23,Crédito Evento B3,JCP BBDC3,01/11/2023,Recebido,3.96,C
20,Transferência recebida,Transferência recebida,01/11/2023,Recebido,8000.00,C
19,Cashback,Cashback,02/11/2023,Recebido,2.10,C
17,Crédito B3,Liquidacao,03/11/2023,Recebido,7.27,C
15,Cred Pontos Cashback Conta,Cashback,03/11/2023,Recebido,62.50,C
12,Crédito Evento B3,Dividendos CXSE3,06/11/2023,Recebido,284.50,C
7,Cred Pontos Cashback Conta,Cashback,07/11/2023,Recebido,57.50,C
5,Crédito B3,Liquidacao,09/11/2023,Recebido,934.93,C
4,Crédito B3,Liquidacao,10/11/2023,Recebido,1735.92,C
21,Pagamento efetuado,SINGULAR,01/11/2023,Educação,942.85,D


In [ ]:
pagina = gc.open('Planilha para Controle Financeiro 2021 em diante').worksheet('extratoInter')

In [ ]:
pagina.clear()

{'spreadsheetId': '1chQdqBEPA_E_5eBWSP2j0KvEwBWZxzjEPJv8oyTSNPk',
 'clearedRange': 'extratoInter!A1:Z1000'}

In [ ]:
pagina.update([resumo.columns.values.tolist()] + resumo.values.tolist())

/home/yair/.local/lib/python3.7/site-packages/gspread/worksheet.py:1049: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  msg_deprecated="method signature will change to: 'Worksheet.update(value = [[]], range_name=)'"


{'spreadsheetId': '1chQdqBEPA_E_5eBWSP2j0KvEwBWZxzjEPJv8oyTSNPk',
 'updatedRange': 'extratoInter!A1:F25',
 'updatedRows': 25,
 'updatedColumns': 6,
 'updatedCells': 150}

In [ ]:
resultado = final.loc[final['classe'].isin(marcador)]

In [ ]:
resultado

,Data Lançamento,Histórico,Descrição,Valor,tipo,onde,classe
21,01/11/2023,Pagamento efetuado,COLEGIO TECNICO SINGULAR,942.85,D,SINGULAR,Educação
22,01/11/2023,Pix enviado,Ingrid Horodynski Donadelli,100.00,D,Ingrid,Mesada
18,02/11/2023,Compra Inter Cel,Inter Pre 10gb Mensal,30.00,D,Celular Gui,Celular
14,04/11/2023,Pix enviado,Carlos Alberto Figaro Bertin,20.00,D,Bertin,Carro
8,07/11/2023,Pix enviado,Ingrid Horodynski Donadelli,60.00,D,Ingrid,Mesada
6,08/11/2023,Pix enviado,Ingrid Horodynski Donadelli,25.00,D,Ingrid,Mesada
3,10/11/2023,Pix enviado,Lourdes Da Conceicao Gomes Carvalho,82.00,D,Lourdes,Med/Dent/Psic


In [ ]:
pagina1.clear()
pagina1.update([resultado.columns.values.tolist()] + resultado.values.tolist())

NameError: name 'pagina1' is not defined

In [ ]:
resultado

,Histórico,onde,Data Lançamento,classe,Valor,tipo
2,Pix enviado,Lourdes,05/09/2023,Med/Dent/Psic,82.0,D
7,Compra Inter Cel,Celular Gui,11/09/2023,Celular,30.0,D
14,Pix enviado,Ingrid,15/09/2023,Mesada,70.0,D
29,Pix enviado,Demerge,22/09/2023,Educação,34.9,D
28,Pix enviado,Lourdes,22/09/2023,Med/Dent/Psic,82.0,D
30,Pix enviado,Fernando,25/09/2023,Med/Dent/Psic,350.0,D
34,Pix enviado,Ingrid,28/09/2023,Mesada,20.0,D


In [ ]:

##########################################################################################################

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

# scope = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']

# credentials = ServiceAccountCredentials.from_json_keyfile_name('destrincha-fatura.json',scope)

# gc = gspread.authorize(credentials)

#########################################################################################################

# pagina0 = gc.open('Extrato Inter 2023').get_worksheet(0)
# #
# pagina0.clear()
# pagina0.update([final.columns.values.tolist()] + final.values.tolist())

# #print(final.groupby('onde').sum())#

